In [18]:
import pandas as pd
import numpy as np

##### Будем работать с датасетом по оттоку клиентов из банка https://www.kaggle.com/datasets/shubh0799/churn-modelling, но датасет из себя будет представлять две таблицы:

1. Личные данные клиента

  A. CustomerId - Уникальный идентификатор клиента

  B. Surname - Фамилия клиента

  C. Geography - Из какой страны клиент

  D. Gender - Пол клиента

  E. Age - Возраст клиента

  F. EstimatedSalary - Предположительная зарплата клиента

2. Данные по поведению клиента в банке

  A. CustomerId - Уникальный идентификатор клиента

  B. CustomerId - Уникальный идентификатор клиента

  C. Tenure - Сколько лет человек является клиентом банка

  D. Balance - Баланс счета

  E. NumOfProducts - Количество открытых продуктов

  F. HasCrCard - Есть ли у клиента кредитная карта

  G. IsActiveMember - Является ли клиент активные участником
  
  H. Exited - Уйдет ли человек в отток

In [ ]:
users = pd.read_csv('users.csv', sep=';')
users.head()

In [ ]:
users.shape

#### Создание новых признаков

In [ ]:
users['new_feature'] = 0
users.head()

In [ ]:
users['Age (days)'] = users['Age'] * 365
users.head()

In [ ]:
for i, row in users.iloc[:2].iterrows():
    print(row)
    print('__' * 30)

In [ ]:
age_days = []

for i, row in users.iterrows():
    age_days.append(row['Age'] * 365)

age_days[:10]

In [ ]:
users['Age (days) 2'] = age_days
users.head()

In [ ]:
def age_to_days(x):
    return x * 365

users['Age (days) 3'] = users['Age'].apply(age_to_days)
users.head()

In [ ]:
import time
from tqdm import tqdm
tqdm.pandas()


def age_to_days(x):
    time.sleep(0.001)
    return x * 365

users['Age'].progress_apply(age_to_days)

#### Удаление признаков

In [ ]:
users.drop(columns='new_feature')
users.head()

In [ ]:
users = users.drop(columns='new_feature')
users.head()

In [ ]:
users['new_feature'] = 0

In [ ]:
users.drop(columns='new_feature', inplace=True)
users.head()

In [ ]:
users.drop(columns=['Age (days)', 'Age (days) 2', 'Age (days) 3'], inplace=True)
users.head()

#### Изменение существующих признаков
#### .loc

In [ ]:
users['target'] = 0
users.head()

In [ ]:
users.loc[users['Geography'] == 'France']

In [ ]:
users.loc[users['Geography'] == 'France', 'target']

In [ ]:
users[users['Geography'] == 'France']['target'] = 1
users.head()

<ipython-input-19-b763340dfd50>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users[users['Geography'] == 'France']['target'] = 1

In [ ]:
users.loc[users['Geography'] == 'France', 'target'] = 1
users.head()

##### .replace

In [ ]:
users['Gender'].replace({'Female': 'F', 'Male': 'M'}, inplace=True)
users.head()

### Методы агрегации

In [ ]:
users['Age'].agg(['min', 'max'])

In [ ]:
users.agg({
    'Age': ['min', 'max'],
    'EstimatedSalary': 'mean'
})

In [ ]:
users.agg(
    min_age=('Age', 'min'),
    max_age=('Age', 'max'),
    mean_salary=('EstimatedSalary', 'mean')
)

#### Методы объединения

In [ ]:
bank = pd.read_csv('bank.csv', sep=';')
bank.head()

In [ ]:
bank.shape

In [ ]:
merged = users.merge(bank, left_on='CustomerId', right_on='CustomerId')
merged.head()

In [ ]:
users_id = users.set_index('CustomerId')
users_id.head()

In [ ]:
bank_id = bank.set_index('CustomerId')
bank_id.head()

In [ ]:
bank_id.join(users_id).head()

In [ ]:
bank_id.join(users_id).reset_index().head()

In [ ]:
bank.shape

### Атрибут how


In [ ]:
toy_df1 = pd.DataFrame({
    'col_1': [1, 2, 3],
    'col_2': [9, 9, 9]
})

toy_df2 = pd.DataFrame({
    'col_1': [3, 4],
    'col_3': [0, 0]
})

display(toy_df1, toy_df2)

In [ ]:
toy_df1.merge(toy_df2, how='left')

In [ ]:
toy_df1.merge(toy_df2, how='right')

In [ ]:
toy_df1.merge(toy_df2, how='inner')

In [ ]:
toy_df1.merge(toy_df2, how='outer')

In [ ]:
merged_left = bank.merge(users, on='CustomerId', how='left')
merged_left.shape

In [ ]:
merged_left.isna().sum()

In [ ]:
merged_left[merged_left['Age'].isna()]

In [ ]:
users[users['CustomerId'] == 15682355]

#### right

In [ ]:
merged_right = bank.merge(users, on='CustomerId', how='right')
merged_right.shape

In [ ]:
merged_right.isna().sum()

In [ ]:
merged_right[merged_right['CreditScore'].isna()]

In [ ]:
bank[bank['CustomerId'] == 15611325]

#### inner

In [ ]:
merged_inner = bank.merge(users, on='CustomerId', how='inner')
merged_inner.shape

In [ ]:
merged_inner.isna().sum()

#### outer

In [ ]:
merged_outer = bank.merge(users, on='CustomerId', how='outer')
merged_outer.shape

In [ ]:
merged_outer.isna().sum()

#### Методы группировок
##### groupby

In [ ]:
toy_df = pd.DataFrame({
    'client_id': [1, 2, 2, 3, 1, 1],
    'item': ['chocolate', 'cheese', 'ham', 'candy', 'chair', 'book'],
    'price': [68, 280, 302, 39, 2099, 1089]
})

toy_df

In [ ]:
grouped = toy_df.groupby('client_id')
grouped

In [ ]:
grouped.groups

In [ ]:
grouped.agg({'price': ['sum', 'min', 'max']})

In [ ]:
users.groupby('Geography').agg({'Age': ['mean'], 'EstimatedSalary': ['min']})

#### pivot_table

In [ ]:
toy_df.pivot_table(index='client_id',
                   values='price',
                   aggfunc='sum')

In [ ]:
users.pivot_table(index='Geography',
                  aggfunc={'Age': ['mean'], 'EstimatedSalary': 'min'})

In [ ]:
users.pivot_table(index='Geography',
                  columns='Gender', 
                  values='EstimatedSalary',
                  aggfunc='mean',
                  margins=True,
                  margins_name='Total')

#### crosstab

In [ ]:
pd.crosstab(index=users['Geography'],
            columns=users['Gender'])

In [ ]:
pd.crosstab(index=users['Geography'],
            columns=users['Gender'],
            values=users['EstimatedSalary'],
            aggfunc='mean')

In [ ]:
pd.crosstab(index=users['Geography'],
            columns=users['Gender'],
            normalize='all')

In [ ]:
pd.crosstab(index=users['Geography'],
            columns=users['Gender'],
            normalize='index')

In [ ]:
pd.crosstab(index=users['Geography'],
            columns=users['Gender'],
            normalize='columns')

#### Встроенные визуализации

In [ ]:
users['Age'].hist();

In [ ]:
data = users.groupby('Gender').count()['Age']
data.name = 'Gender'
data

In [ ]:
data.plot.pie(y='Gender');

In [ ]:
users.iloc[:100].plot.scatter(x='Age', y='EstimatedSalary');

In [ ]:
data = bank.groupby('Tenure').count()['Balance']
data.name = 'num_clients'
data

In [ ]:
data.plot.bar(width=0.8);